In [1]:
# This Code pushes data to my MongoDb Database

import requests
from pymongo import MongoClient
import xml.etree.ElementTree as ET
import logging

connection_string = "mongodb+srv://#########################################################################/"
client = MongoClient(connection_string)

/Users/punyakn/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Database name
db = client['DataAnalysis_SS']

In [3]:
# Function to retrieve data from XML and store it in MongoDB
def store_xml_data_in_mongodb(xml_url, CausesOfDeath, limit=1200):
    response = requests.get(xml_url)
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        collection = db[CausesOfDeath]
        count = 0
        for row in root.findall('.//row'):
            if count >= limit:
                break
            data = {}
            for field in row:
                data[field.tag] = field.text
            collection.insert_one(data)
            count += 1
        print("XML data stored successfully in MongoDB.")
    else:
        print("Failed to retrieve XML data.")

In [5]:
# Function to retrieve data from JSON and store it in MongoDB
def store_json_data_in_mongodb(json_url, DrugOverdose, limit=1200):
    response = requests.get(json_url)
    if response.status_code == 200:
        try:
            json_data = response.json()
            if 'data' in json_data:
                data_records = json_data['data'][:limit]
                collection = db[DrugOverdose]
                for record in data_records:
                    if len(record) >= 14:
                        desired_fields = {
                            "INDICATOR": record[8],
                            "PANEL": record[9],
                            "PANEL_NUM": record[10],
                            "UNIT": record[11],
                            "UNIT_NUM": record[12],
                            "STUB_NAME": record[13],
                            "STUB_NAME_NUM": record[14],
                            "STUB_LABEL": record[15],
                            "STUB_LABEL_NUM": record[16],
                            "YEAR": record[17],
                            "YEAR_NUM": record[18],
                            "AGE": record[19],
                            "AGE_NUM": record[20],
                            "ESTIMATE": record[21]
                        }
                        collection.insert_one(desired_fields)
                print("JSON data stored successfully in MongoDB.")
            else:
                print("JSON data does not contain 'data' key.")
        except Exception as e:
            print("Error occurred while processing JSON data:", e)
    else:
        print("Failed to retrieve JSON data.")


In [6]:

# Function to retrieve data from XML and store it in MongoDB for a Suicide
def store_additional_xml_data_in_mongodb(xml_url, Suicide, limit=1200):
    try:
        response = requests.get(xml_url)
        response.raise_for_status()
        root = ET.fromstring(response.content)
        collection = db[Suicide]
        count = 0
        for row in root.findall('.//row'):
            if count >= limit:
                break
            data = {elem.tag: elem.text for elem in row}
            collection.insert_one(data)
            count += 1
        logging.info("Suicide XML data stored successfully in MongoDB.")
    except requests.RequestException as e:
        logging.error(f"Failed to retrieve additional XML data: {e}")
    except ET.ParseError as e:
        logging.error(f"XML parsing failed: {e}")
    except Exception as e:
        logging.error(f"Error storing additional data in MongoDB: {e}")


In [7]:
# Store Suicide XML data in MongoDB 
store_additional_xml_data_in_mongodb("https://data.cdc.gov/api/views/9j2v-jamp/rows.xml?accessType=DOWNLOAD", "Suicide")

# Store XML data in MongoDB
store_xml_data_in_mongodb("https://data.cdc.gov/api/views/bi63-dtpu/rows.xml?accessType=DOWNLOAD", "CausesOfDeath")

# Store JSON data in MongoDB
store_json_data_in_mongodb("https://data.cdc.gov/api/views/95ax-ymtc/rows.json?accessType=DOWNLOAD", "DrugOverdose")


XML data stored successfully in MongoDB.
JSON data stored successfully in MongoDB.
